In [3]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import pandas as pd
%matplotlib inline
from time import clock
import powerlaw as pl
import scipy.special
import scipy.stats as sp

# PowerLaw Fit

In [4]:
def plot_dataset(arr):
    x = np.array(arr)
    q = np.unique(x)

    n = len(x)
    results = pl.Fit(x)
    xmin = results.power_law.xmin
    alpha = results.power_law.alpha
    
    if sum(x-np.floor(x)):
        c = np.r_[np.sort(x), np.arange(n, 0, -1)/n].reshape(2, n)
        q = np.sort(x[x >= xmin])
        cf = np.r_[q, (q/xmin)**(1-alpha)].reshape(2, len(q))
        cf[1, :] = cf[1, :] * max(c[1, c[0, :] >= xmin])
    else:
        c = np.histogram(x, np.r_[q,q[-1]+1])[0]/float(n)
        c = np.r_[np.r_[q,q[-1]+1],1-np.r_[0,np.cumsum(c)]].reshape(2,len(q)+1)
        c = c[:,c[1,:]>=1e-10]
        cf = (np.r_[xmin:q[-1]+1]**-alpha)/scipy.special.zeta(alpha, xmin)
        cf = np.r_[np.r_[xmin:q[-1]+2], 1 - np.r_[0, np.cumsum(cf)]].reshape(2, q[-1]+2-xmin)
        cf[1, :] = cf[1, :] * c[1, c[0, :] == xmin]

    plt.xticks()
    plt.loglog(c[0,:],c[1,:],'bo', markersize=10, markerfacecolor='w')
    plt.loglog(cf[0,:],cf[1,:],label=alpha)
    plt.legend(loc = 'lower left')
    return alpha, xmin

# Measure

In [5]:
def balanced_degree(g):
    degree = np.array(g.degree().values())
    return np.var(degree)

def balanced_closness(g):
    closness= np.array(nx.closeness_centrality(g).values())
    return np.var(closness)

def balanced_betweenness_node(g):
    betweenness_node = np.array(nx.betweenness_centrality(g).values())
    return np.var(betweenness_node)

def balanced_betweenness_edges(g):
    betweennes_edges = np.array(nx.centrality.edge_betweenness_centrality(g).values())
    return np.var(betweennes_edges)

def spectral_gap(g):
    eig = np.sort(nx.adjacency_spectrum(g))[-2:]
    eig_n_1,eig_n = eig.real
    return eig_n - eig_n_1

def natural_connectivity(g):
    eig = nx.adjacency_spectrum(g).real
    return np.log(np.mean(np.exp(eig)))

def weighted_spectral_distribution(g,N=4):
    L = nx.normalized_laplacian_matrix(g).todense()
    eig = 1 - np.linalg.eigh(L)[0]
    return np.sum(np.power(eig,N))

def network_ciriticality(g):
    L = nx.laplacian_matrix(g).todense()
    trace = np.trace(np.linalg.pinv(L))
    N = g.number_of_nodes()
    return (2.0/(N-1))*trace

def effective_graph_resistance(g):
    N = g.number_of_nodes()
    eig = np.sort(nx.laplacian_spectrum(g))
    eig = eig[1:]
    return N*(np.sum(1.0/eig))

def effective_graph_conductance(g):
    N = g.number_of_nodes()
    R_G = effective_graph_resistance(g)
    return (N-1)/R_G

def flow_robustness(G,n):
    FR = 0
    for c_g in nx.connected_component_subgraphs(G):
        C_i = len(c_g)
        FR+= C_i*(C_i - 1)        
    return FR/float(n*(n-1))

def energy(G):
    return np.sum(np.abs(nx.adjacency_spectrum(G)))

def sum_of_flow_robustness_betweenness(G1):
    if len(G1) == 0:
        print "Graph is empty"
    else:
        G = G1.copy()
        N = G.number_of_nodes()
        SF = flow_robustness(G,N) 
        #print SF
        for i in range(N):
            betweenness = nx.betweenness_centrality(G)
            betweenness = sorted(betweenness.items(),key=lambda(x,y): y , reverse=True)
            G.remove_node(betweenness[0][0])
            SF += flow_robustness(G,N)
            #print SF
        return SF

def sum_of_flow_robustness_degree(G1):
    if len(G1)==0:
        print "Graph is empty"
    else:
        G = G1.copy()
        N = G.number_of_nodes()
        SF = flow_robustness(G,N)
        #print SF
        for i in range(N):
            betweenness = nx.degree(G)
            betweenness = sorted(betweenness.items(),key=lambda(x,y): y , reverse=True)
            G.remove_node(betweenness[0][0])
            SF += flow_robustness(G,N)
            #print SF
        return SF

def sum_of_flow_robustness_closeness(G1):
    if len(G1)==0:
        print "Graph is empty"
    else:
        G = G1.copy()
        N = G.number_of_nodes()
        SF = flow_robustness(G,N)
        #print SF
        for i in range(N):
            betweenness = nx.closeness_centrality(G)
            betweenness = sorted(betweenness.items(),key=lambda(x,y): y , reverse=True)
            G.remove_node(betweenness[0][0])
            SF += flow_robustness(G,N)
            #print SF
        return SF

def path2elements(path):
    set_elem = set()
    edge= []
    for i in range(0,len(path)-1,1):
        if path[i] <= path[i+1]:
            edge.append((path[i],path[i+1]))
        elif path[i] > path[i+1]:
            edge.append((path[i+1],path[i]))
            
    #x1  = path[0]
    #x2  = path[-1]
    #path.remove(x1)
    #path.remove(x2)
    
    set_elem.update(path[1:-1])
    set_elem.update(edge)
    return set_elem

def p_div(path , sh_path):
    p = path2elements(path)
    s_p = path2elements(sh_path)
    intersection = s_p.intersection(p)
    return 1 - (len(intersection)/len(s_p))

def k_diverse(g , s , d , h , k):
    
    diverse_path = []               #empty ordered list
    selected_elements = set()       # empty set
    
    sh_path = nx.shortest_path(g,s,d)
    selected_elements.update(sh_path)
    
    for path in nx.all_simple_paths(g , s , d , h):
        diverse_path.append((path , p_div(path , sh_path) , len(path)))
        selected_elements.union(path2elements(path))
    
    diverse_path.sort(key=lambda(x,y,z):(z),reverse= False)
    diverse_path.sort(key=lambda(x,y,z):(y),reverse= True )
    
    return diverse_path[::k]

def effective_path_diversity( g , s , d , h , k , landa=1):
    diverse_paths = k_diverse(g , s , d , h , k)
    epd = []
    for path in diverse_paths:
        if path[1] !=0:
            epd.append(path[1])
    k_s_d = sum(epd)
    return 1 - np.exp(-landa*k_s_d)

def total_diversity_path(G1,h,k,landa=1):
    G = nx.convert_node_labels_to_integers(G1)
    N = G.number_of_nodes()
    TGD = []
    i=0
    for node1 in range(N):
        for node2 in range(node1+1,N):
            TGD.append(effective_path_diversity(G , node1 , node2 , h , k , landa))
    return np.mean(TGD)

def effective_diversity_path(G1,source,destination,landa=1):
    G = nx.convert_node_labels_to_integers(G1)
    sh_path = nx.shortest_path(G,source,destination)
    
    edge_sh = []
    for i in range(0,len(sh_path)-1,1):
        edge_sh.append((sh_path[i],sh_path[i+1]))
        edge_sh.append((sh_path[i+1],sh_path[i]))
    #print edge_sh
    #sh_path.remove(sh_path[0])
    #sh_path.remove(sh_path[-1])
    #print sh_path
    len_sh_path=len(sh_path) + len(edge_sh)
    
    k_s_d = 0
    
    for path in nx.all_simple_paths(G,source,destination,10): 
        edge = []
        for i in range(0,len(path)-1,1):
            edge.append((path[i],path[i+1]))
            edge.append((path[i+1],path[i]))
        #print path
        #print edge
        #path.remove(path[0])
        #path.remove(path[-1])
        intersect = len(np.intersect1d(sh_path, path)) + len(np.intersect1d(edge_sh, edge))
        path_dive = 1 - ((intersect)/float(len_sh_path))
        
        k_s_d +=path_dive
    #print k_s_d
    #print np.exp(-(landa*k_s_d))
        
    EDP = 1 - np.exp(-(landa*k_s_d))
    return EDP

# Attack

In [6]:
def betweenness_attack(G1):
    G = G1.copy()
    N = G.number_of_nodes()
    sfrb =[]
    en =[]
    for i in range(N-2):
        betweenness = nx.betweenness_centrality(G)
        betweenness = sorted(betweenness.items(),key=lambda(x,y): y , reverse=True)
        G.remove_node(betweenness[0][0])
        sfrb.append(sum_of_flow_robustness_betweenness(G))
        en.append(energy(G))
    return sfrb,en

def degree_attack(G1):
    G = G1.copy()
    N = G.number_of_nodes()
    sfrd =[]
    en =[]
    #
    for i in range(N-2):
        betweenness = nx.degree(G)
        betweenness = sorted(betweenness.items(),key=lambda(x,y): y , reverse=True)
        G.remove_node(betweenness[0][0])
        sfrd.append(sum_of_flow_robustness_degree(G))
        en.append(energy(G))
    return sfrd,en

def closeness_attack(G1):
    G = G1.copy()
    N = G.number_of_nodes()
    sfrc =[]
    en =[]
    for i in range(N-2):
        betweenness = nx.closeness_centrality(G)
        betweenness = sorted(betweenness.items(),key=lambda(x,y): y , reverse=True)
        G.remove_node(betweenness[0][0])
        sfrc.append(sum_of_flow_robustness_closeness(G))
        en.append(energy(G))
    return sfrc,en

# Apollonian

In [7]:
def det_Apollonian(Tmax):    
    D = np.array([0, 1, 2]).reshape(1,3)
    
    #colors = ['red','blue','green','yellow','cyan','magenta','black']*3
    E = np.zeros((3, 2))
    E[0,:] = np.array([0  ,0])
    E[1,:] = np.array([1,2])
    E[2,:] = np.array([2  ,0])
    nodeidx = E.shape[0]

    #plt.figure(figsize=(15,15))
    XY0 = E[D[0]]                                     
    #plt.plot(XY0[:,0],XY0[:,1],'bo',markersize=2)
    #plt.plot(XY0[:,0],XY0[:,1],'r',alpha = 0.25,lw=1)
    #plt.plot(XY0[[0,-1],0],XY0[[0,-1],1],'r',alpha=0.25,lw=1)
    #plt.xlim(-0.1,2.1)
    #plt.ylim(-0.1,2.1)
    
    T = nx.Graph()
    T.add_node(0)

    for i in range(Tmax):
        if i == 0:
            nTri = 0
        else:
            nTri += 3**(i-1)
        
        for c in range(3**(i)):
            lidx = D.shape[0]
            D = np.vstack((D,[D[nTri+c,0],D[nTri+c,1],nodeidx]))
            D = np.vstack((D,[D[nTri+c,1],D[nTri+c,2],nodeidx]))
            D = np.vstack((D,[D[nTri+c,0],D[nTri+c,2],nodeidx]))
            T.add_edges_from([(nTri + c, D.shape[0]-1),(nTri + c, D.shape[0]-2),(nTri + c, D.shape[0]-3)])
            
            
            XY = E[D[nTri + c,:]]

            x0,y0 = XY[0]
            xc,yc = (XY[1,0] + XY[2,0])/2.0 , (XY[1,1] + XY[2,1])/2.0
            x = x0 + 2.0/3.0*(xc-x0)
            y = y0 + 2.0/3.0*(yc-y0)
            E = np.vstack((E,np.array([x,y])))
            nodeidx = E.shape[0]
            
            XY0 = E[D[nTri + c]]
            #plt.plot(E[-1,0],E[-1,1],'bo',markersize=2)
            #for j in range(3):
            #    plt.plot([XY0[j,0],E[-1,0]],[XY0[j,1],E[-1,1]],'r',alpha= 0.25,lw=1)
    A = np.zeros((E.shape[0],E.shape[0]))
    for d in D:
        A[np.ix_(d,d)] = 1
    for i in range(A.shape[0]):
        A[i,i] = 0
    #print A
    plt.axis('off')
    g = nx.from_numpy_matrix(A)
    g.name = "deterministic_Apollonian"
    return g,T

def rnd_Apollonian(Tmax=100):
    A          = np.zeros((Tmax + 3,Tmax + 3))
    A[0:3,0:3] = np.ones((3,3)) - np.eye(3)
    XY0        = np.array([[0    ,0],
                           [1/2.0,1.],
                           [1    ,0]])
    
    E          = np.zeros((Tmax + 3 , 2))
    E[0:3,:]   = XY0
    D          = np.zeros((1 , 3)    ,dtype='int32')      # Matrix for triangle
    D[0]       = np.array([0 , 1 , 2],dtype='int32')
    AD         = D.copy()
    
    T = nx.Graph()
    T.add_node(0)

    #plt.figure(figsize=(15,15),dpi=72)
    #plt.plot(XY0[:,0],XY0[:,1],'bo',markersize=2)
    #plt.plot(XY0[:,0],XY0[:,1],'r',alpha=.25)
    #plt.plot(XY0[[0,-1],0],XY0[[0,-1],1],'r',alpha=.25)
    #plt.ylim(-.1,1.1)
    #plt.xlim(-.1,1.1)
    
    for t in range(Tmax):
        Lrow             = AD.shape[0]
        Di               = np.random.randint(Lrow)
        XY               = E[AD[Di]]

        x0,y0 = XY[0]
        xc,yc = (XY[1,0] + XY[2,0])/2.0 , (XY[1,1] + XY[2,1])/2.0
        x = x0 + 2.0/3.0*(xc-x0)
        y = y0 + 2.0/3.0*(yc-y0)
        
        XY0              = np.array([x , y])
        E[t + 3]         = XY0
        
        A[[t + 3]*3,AD[Di]] = 1
        A[[AD[Di]],[t+3]*3] = 1

        D=np.vstack((D,[AD[Di,0],AD[Di,1],t+3]))
        D=np.vstack((D,[AD[Di,1],AD[Di,2],t+3]))
        D=np.vstack((D,[AD[Di,0],AD[Di,2],t+3]))
        
        AD=np.vstack((AD,[AD[Di,0],AD[Di,1],t+3]))
        AD=np.vstack((AD,[AD[Di,1],AD[Di,2],t+3]))
        AD=np.vstack((AD,[AD[Di,0],AD[Di,2],t+3]))
        
        
        idx = np.where(np.all(D == AD[Di],axis=1))[0][0]
        T.add_edges_from([(idx, D.shape[0]-1), (idx, D.shape[0]-2), (idx, D.shape[0]-3)])
        
        XY = E[AD[Di]]
        AD=np.delete(AD,Di,axis=0)
        
        #plt.plot(E[t + 3,0],E[t + 3,1],'bo',markersize=2)
        #for j in range(3):
        #    plt.plot([XY[j,0],E[t + 3 , 0]],[XY[j,1],E[t + 3,1]],'r',alpha= 0.25)
    #plt.axis('off')
    g = nx.from_numpy_matrix(A)
    g.name = "random_Apollonian"
    return g,T


def rnd_ApollonianTri(Tmax=100):
    A          = np.zeros((Tmax + 3,Tmax + 3))
    A[0:3,0:3] = np.ones((3,3)) - np.eye(3)
    XY0        = np.array([[0    ,0],
                           [1/2.0,1.],
                           [1    ,0]])
    
    E          = np.zeros((Tmax + 3 , 2))
    E[0:3,:]   = XY0
    D          = np.zeros((1 , 3)    ,dtype='int32')      # Matrix for triangle
    D[0]       = np.array([0 , 1 , 2],dtype='int32')
    AD         = D.copy()
    
    T = nx.Graph()
    T.add_node(0)

    #plt.figure(figsize=(15,15),dpi=72)
    #plt.plot(XY0[:,0],XY0[:,1],'bo',markersize=2)
    #plt.plot(XY0[:,0],XY0[:,1],'r',alpha=.25)
    #plt.plot(XY0[[0,-1],0],XY0[[0,-1],1],'r',alpha=.25)
    #plt.ylim(-.1,1.1)
    #plt.xlim(-.1,1.1)
    
    for t in range(Tmax):
        Lrow             = AD.shape[0]
        area_of_tri      = []
        for i in range(Lrow):
            idx = AD[i]
            p1,p2,p3 = E[idx]
            area = abs((p1[0]*(p3[1]-p2[1]) + p2[0]*(p1[1] - p3[1]) + p3[0]*(p1[1] - p2[1]))/2.0)
            area_of_tri.append(area)
        area_of_tri = np.array(area_of_tri)
        s = np.sum(area_of_tri)
        p = area_of_tri / s
            
        Di               = np.random.choice(range(Lrow),size=1,replace=True,p=p)[0]        # select triangular
        XY               = E[AD[Di]]

        x0,y0 = XY[0]
        xc,yc = (XY[1,0] + XY[2,0])/2.0 , (XY[1,1] + XY[2,1])/2.0
        x = x0 + 2.0/3.0*(xc-x0)
        y = y0 + 2.0/3.0*(yc-y0)
        
        XY0              = np.array([x , y])
        E[t + 3]         = XY0
        
        A[[t + 3]*3,AD[Di]] = 1
        A[[AD[Di]],[t+3]*3] = 1

        D=np.vstack((D,[AD[Di,0],AD[Di,1],t+3]))
        D=np.vstack((D,[AD[Di,1],AD[Di,2],t+3]))
        D=np.vstack((D,[AD[Di,0],AD[Di,2],t+3]))
        
        AD=np.vstack((AD,[AD[Di,0],AD[Di,1],t+3]))
        AD=np.vstack((AD,[AD[Di,1],AD[Di,2],t+3]))
        AD=np.vstack((AD,[AD[Di,0],AD[Di,2],t+3]))
        
        
        idx = np.where(np.all(D == AD[Di],axis=1))[0][0]
        T.add_edges_from([(idx, D.shape[0]-1), (idx, D.shape[0]-2), (idx, D.shape[0]-3)])
        
        XY = E[AD[Di]]
        AD=np.delete(AD,Di,axis=0)
        
        #plt.plot(E[t + 3,0],E[t + 3,1],'bo',markersize=2)
        #for j in range(3):
        #    plt.plot([XY[j,0],E[t + 3 , 0]],[XY[j,1],E[t + 3,1]],'r',alpha= 0.25)
    #plt.axis('off')
    g = nx.from_numpy_matrix(A)
    g.name = "Tri_Apollonian"
    return g,T

def evolution_Apollonian(Tmax=100,q=0.5):
    A          = np.zeros((Tmax + 3,Tmax + 3))
    A[0:3,0:3] = np.ones((3,3)) - np.eye(3)
    XY0        = np.array([[0    ,0],
                           [1/2.0,1.],
                           [1    ,0]])
    
    E = np.zeros((3, 2))
    E[0,:] = np.array([0  ,0])
    E[1,:] = np.array([0.5,1])
    E[2,:] = np.array([1  ,0])
    
    D          = np.zeros((1 , 3)    ,dtype='int32')      # Matrix for triangle
    D[0]       = np.array([0 , 1 , 2],dtype='int32')
    AD         = D.copy()
    
    T = nx.Graph()
    T.add_node(0)

    #plt.figure(figsize=(15,15),dpi=72)
    #plt.plot(XY0[:,0],XY0[:,1],'bo',markersize=2)
    #plt.plot(XY0[:,0],XY0[:,1],'r',alpha=.25)
    #plt.plot(XY0[[0,-1],0],XY0[[0,-1],1],'r',alpha=.25)
    #plt.ylim(-.1,1.1)
    #plt.xlim(-.1,1.1)
    idx = 3
    for t in range(Tmax):
        Lrow             = AD.shape[0]
        rand = np.random.binomial(Lrow,q)
        for i in range(rand):
            Di               = np.random.randint(Lrow)
            XY               = E[AD[Di]]

            x0,y0 = XY[0]
            xc,yc = (XY[1,0] + XY[2,0])/2.0 , (XY[1,1] + XY[2,1])/2.0
            x = x0 + 2.0/3.0*(xc-x0)
            y = y0 + 2.0/3.0*(yc-y0)

            E = np.vstack((E,np.array([x,y])))

            D=np.vstack((D,[AD[Di,0],AD[Di,1],idx]))
            D=np.vstack((D,[AD[Di,1],AD[Di,2],idx]))
            D=np.vstack((D,[AD[Di,0],AD[Di,2],idx]))

            AD=np.vstack((AD,[AD[Di,0],AD[Di,1],idx]))
            AD=np.vstack((AD,[AD[Di,1],AD[Di,2],idx]))
            AD=np.vstack((AD,[AD[Di,0],AD[Di,2],idx]))
            
            idx1 = np.where(np.all(D == AD[Di],axis=1))[0][0]
            T.add_edges_from([(idx1, D.shape[0]-1), (idx1, D.shape[0]-2), (idx1, D.shape[0]-3)])

            XY = E[AD[Di]]
            AD=np.delete(AD,Di,axis=0)

            #plt.plot(E[-1,0],E[-1,1],'bo',markersize=2)
            #for j in range(3):
            #    plt.plot([XY[j,0],E[-1, 0]],[XY[j,1],E[-1,1]],'r',alpha= 0.25)
            #Lrow = AD.shape[0]
            idx +=1
        
    A = np.zeros((E.shape[0],E.shape[0]))
    for d in D:
        A[np.ix_(d,d)] = 1
    for i in range(A.shape[0]):
        A[i,i] = 0
    print idx
    #plt.axis('off')
    g = nx.from_numpy_matrix(A)
    g.name = "Evolution_Apollonian"
    
    return g,T


In [7]:
class genetic():
    """
    Parameter:
    g: Graph
    p: percentage of rewiring
    popLen: number of population
    itrIn : number of iteration for reapeted result
    itrOut: number of outer iteration
    
    outPut:
    fit: fitness for best solution
    g: optimized Graph
    rmv_idx: index for removed edges
    add_idx: index for added edges
    """
    def __init__(self , g , p , popLen , itrIn , itrOut):
        self.p                 =p                                       # rewiring percentage
        self.g                 =g.copy()
        self.g_c               =nx.complement(self.g)
        self.number_of_edges   =self.g.number_of_edges()                     
        self.number_of_edges_c =self.g_c.number_of_edges()
        self.m                 =int(self.p * self.number_of_edges)      # solution length
        self.edges             =np.array(self.g.edges())                     
        self.edges_c           =np.array(self.g_c.edges())
        self.popLen            =popLen                                  # number of population
        self.itrIn             =itrIn
        self.itrOut            =itrOut
        self.L                 =nx.laplacian_matrix(self.g).todense()
        self.al                =np.linalg.eigh(self.L)[1].T[1]
        self.al                =np.array(self.al)
        self.al                =np.squeeze(self.al)
    
    # Population
    def population(self):
        alphas_rmv =np.array([self.alpha(edge) for edge in self.g.edges()])
        s = np.sum(alphas_rmv)
        alphas_rmv = s - alphas_rmv
        s = np.sum(alphas_rmv)
        p_rmv = alphas_rmv/float(s)
        rmv_idxs = np.random.choice(np.arange(0,self.number_of_edges),(self.popLen,self.m),replace = True)
        
        alphas_add = np.array([self.alpha(edge) for edge in self.g_c.edges()])
        s = np.sum(alphas_add)
        p_add = alphas_add/float(s)
        add_idxs = np.random.choice(np.arange(0,self.number_of_edges_c),(self.popLen,self.m),replace=True)
        
        return rmv_idxs , add_idxs
    
    # Fitness function

    def compute_Algebraic_connectivity(self, A):
        ei  =np.linalg.eigvalsh(A)
        alg =ei[1]
        return alg
    
    def energy(self,g):
        return np.sum(np.abs(nx.adjacency_spectrum(g).real))
    
    def fitnessE(self , rmv_idxs , add_idxs):
        g_new = self.g.copy()
        leng = rmv_idxs.shape[0]
        fitness = np.zeros(leng, dtype = 'float32')
        for pop in range(leng):
            g_new = self.g.copy()
            leng1 = len(np.unique(rmv_idxs[pop]))
            leng2 = len(np.unique(add_idxs[pop]))
            leng = min([leng1,leng2])

            rmv_idxs_u = np.unique(rmv_idxs[pop])
            add_idxs_u = np.unique(add_idxs[pop])
            g_new.add_edges_from   (self.edges_c[add_idxs_u[:leng]])
            g_new.remove_edges_from(self.edges[rmv_idxs_u[:leng]])
            alg = nx.algebraic_connectivity(g_new)
            if alg == 0:
                fitness[pop] = 0
            else:
                en = self.energy(g_new)
                fitness[pop] = en + alg
        return fitness
    
    def alpha(self ,edge):
        return np.abs(self.al[edge[0]] - self.al[edge[1]])
    
    # Selection
    
    def fitnessSelection(self , rmv_idxs , add_idxs):
        fitness = self.fitnessE(rmv_idxs , add_idxs)
        s = np.sum(fitness)
        newRmv_idxs = rmv_idxs[np.random.choice(np.arange(0,len(rmv_idxs)),self.popLen,replace=True,p=(fitness)/float(s))]
        newAdd_idxs = add_idxs[np.random.choice(np.arange(0,len(add_idxs)),self.popLen,replace=True,p=fitness/float(s))]
        return newRmv_idxs,newAdd_idxs

    def tournamentSelection(self , rmv_idxs , add_idxs):
        newRmv_idxs = np.zeros_like(rmv_idxs)
        newAdd_idxs = np.zeros_like(add_idxs)
        for i in range(0 , self.popLen , 2):
            idxs = np.random.choice(np.arange(0 , self.popLen), 4, replace = False)
            fitness = self.fitnessE(rmv_idxs[idxs] , add_idxs)
            idxs = np.argsort(fitness)[::-1][:2]
            newRmv_idxs[i]   = rmv_idxs[idxs[0]]
            newRmv_idxs[i+1] = rmv_idxs[idxs[1]]

            newAdd_idxs[i]   = add_idxs[idxs[0]]
            newAdd_idxs[i+1] = add_idxs[idxs[1]]
        return newRmv_idxs , newAdd_idxs

    def turncationSelection(self , rmv_idxs , add_idxs):
        f = 0.5
        fitness = self.fitnessE(rmv_idxs , add_idxs)
        l = fitness.shape[0]
        idxs = np.argsort(fitness)[::-1]
        idxs = idxs[:int(f*l)]
        idxs = np.r_[idxs , idxs]
        np.random.shuffle(idxs)
        return rmv_idxs[idxs] , add_idxs[idxs]


    # Crossover
    def uniformCrossover(self , rmv_idxs , add_idxs):
        newRmv_idxs = np.zeros_like(rmv_idxs)
        newAdd_idxs = np.zeros_like(add_idxs)
        newRmv_idxs[0,:] = rmv_idxs[0,:]
        newAdd_idxs[0,:] = add_idxs[0,:]
        for i in range(1 , self.popLen-1 , 2):
            point = np.random.randint(0 , 2 , self.m)
            newRmv_idxs[i,:]   = np.where(point == 1 , rmv_idxs[i] , rmv_idxs[i+1])
            newRmv_idxs[i+1,:] = np.where(point == 1 , rmv_idxs[i+1] , rmv_idxs[i])

            newAdd_idxs[i,:]   = np.where(point == 1 , add_idxs[i] , add_idxs[i+1])
            newAdd_idxs[i+1,:] = np.where(point == 1 , add_idxs[i+1] , add_idxs[i])
        return newRmv_idxs , newAdd_idxs

    def spCrossover(self , rmv_idxs , add_idxs):
        point = np.random.randint(0 , self.m , self.popLen)
        newRmv_idxs = np.zeros_like(rmv_idxs)
        newAdd_idxs = np.zeros_like(add_idxs)
        for i in range(0 , self.popLen , 2):
            newRmv_idxs[i,:point[i]] = rmv_idxs[i  ,:point[i]]
            newRmv_idxs[i,point[i]:] = rmv_idxs[i+1,point[i]:]
            newAdd_idxs[i,:point[i]] = add_idxs[i  ,:point[i]]
            newAdd_idxs[i,point[i]:] = add_idxs[i+1,point[i]:]

            newRmv_idxs[i+1,:point[i+1]] = rmv_idxs[i+1,:point[i+1]]
            newRmv_idxs[i+1,point[i+1]:] = rmv_idxs[i  ,point[i+1]:]
            newAdd_idxs[i+1,:point[i+1]] = add_idxs[i+1,:point[i+1]]
            newAdd_idxs[i+1,point[i+1]:] = add_idxs[i  ,point[i+1]:]
        return newRmv_idxs , newAdd_idxs

    def optimize_graph(self , rmv_idxs , add_idxs, idx):
        g_n = self.g.copy()
        leng1 = len(np.unique(rmv_idxs[idx]))
        leng2 = len(np.unique(add_idxs[idx]))
        leng = min([leng1,leng2])
        
        rmv_idxs_u = np.unique(rmv_idxs[idx])
        add_idxs_u = np.unique(add_idxs[idx])
        g_n.add_edges_from   (self.edges_c[add_idxs_u[:leng]])
        g_n.remove_edges_from(self.edges[rmv_idxs_u[:leng]])
        return g_n
    
    def kernel(self,selection='turncationSelection',crossOver='uniformCrossover'):
        li = []
        for i in range(self.itrOut):
            rmv_idxs , add_idxs = self.population()
            avg_fit = 0
            i = 0
            j = 0
            while (i < 10) and (j < self.itrIn):
                avg_fit_pre = avg_fit
                temp_rmv_idxs , temp_add_idxs = getattr(self , selection)(rmv_idxs , add_idxs)
                temp_rmv_idxs , temp_add_idxs = getattr(self , crossOver)(temp_rmv_idxs , temp_add_idxs)
                fit = self.fitnessE(temp_rmv_idxs , temp_add_idxs)
                avg_fit = np.mean(fit)
                if avg_fit > avg_fit_pre:
                    rmv_idxs = temp_rmv_idxs
                    add_idxs = temp_add_idxs
                    print 'avg_fit: ' + str(avg_fit)
                    i = 0
                else:
                    i+=1
                print 'i: ' + str(i)
                j+=1
                print 'j: ' + str(j)
            fit = self.fitnessE(rmv_idxs , add_idxs)
            max_idx = fit.argmax()
            print fit[max_idx]
            print '----------------'
            avg_fit_pre = fit[max_idx]
            li.append((rmv_idxs[max_idx].reshape(1,self.m),add_idxs[max_idx].reshape(1,self.m),fit[max_idx]))
        rmv_idx , add_idx , fit = sorted(li,key = lambda(x,y,z):z,reverse = True)[0]
        return fit ,self.optimize_graph(rmv_idx , add_idx, 0),rmv_idx,add_idx

In [8]:
class optimizeAlgebraicSydney():
    """
    
    """
    def __init__(self , g , p):
        self.p               = p;
        self.g               = g.copy();
        self.g_c             = nx.complement(self.g);
        self.number_of_edges = self.g.number_of_edges();
        self.m               = int(self.p * self.number_of_edges)
        
        
    def alpha(self , g , edge):
        L = nx.laplacian_matrix(g).todense()
        al = np.linalg.eigh(L)[1].T[1]
        al = np.array(al)
        al = np.squeeze(al)
        return np.abs(al[edge[0]] - al[edge[1]])
        
    def OptimizeAlgebraicSydneyFirstAdd(self):
        g_co = self.g.copy()
        g_co_c = self.g_c.copy()
        
        rmv_edgs = np.array(g_co.edges()   , dtype='int32')
        add_edgs = np.array(g_co_c.edges() , dtype='int32')
        
        idx = np.random.choice(np.arange(0,len(add_edgs)),len(rmv_edgs),replace=False)
        add_edgs = add_edgs[idx]

        j=0
        while ((j < self.m) and (j < rmv_edgs.shape[0]) and (j < add_edgs.shape[0])):
            alpha_remove = np.zeros(rmv_edgs.shape[0])   # array for remove edge alpha 
            alpha_add    = np.zeros(add_edgs.shape[0])   # array for add edge alpha

            i = 0
            for rmv_edge in rmv_edgs:
                alpha_remove[i] = self.alpha(g_co , rmv_edge)
                i+=1
            i = 0
            for add_edge in add_edgs:
                alpha_add[i] = self.alpha(g_co , add_edge)
                i+=1

            idx_add    = np.argmax(alpha_add)    # index for edge with maximum alpha
            idx_remove = np.argmin(alpha_remove) # index for edge with minimum alpha

            max_add_edge = add_edgs[idx_add]     # edge with maximum alpha
            min_rmv_edge = rmv_edgs[idx_remove]  # edge with minimum alpha

            g_co.add_edge(*max_add_edge)
            g_co.remove_edge(*min_rmv_edge)

            if not(nx.is_connected(g_co)):
                g_co.add_edge(*min_rmv_edge)
                g_co.remove_edge(*max_add_edge)
                j-=1

            rmv_edgs = rmv_edgs[np.setdiff1d(np.arange(0,rmv_edgs.shape[0]) , idx_remove)]
            add_edgs = add_edgs[np.setdiff1d(np.arange(0,add_edgs.shape[0]) , idx_add   )]    
            j+=1
        return g_co
    
    def OptimizeAlgebraicSydneyFirstRmv(self):
        g_co   = self.g.copy()
        g_co_c = self.g_c.copy()

        rmv_edgs = np.array(g_co.edges()   , dtype='int32')
        add_edgs = np.array(g_co_c.edges() , dtype='int32')
        idx = np.random.choice(np.arange(0,len(add_edgs)),len(rmv_edgs),replace=False)
        add_edgs = add_edgs[idx]

        j = 0
        while((j < self.m) and (j < rmv_edgs.shape[0]) and (j < add_edgs.shape[0])):
            alpha_remove = np.zeros(rmv_edgs.shape[0])   # array for remove edge alpha 
            alpha_add    = np.zeros(add_edgs.shape[0])   # array for add edge alpha

            i = 0
            for rmv_edge in rmv_edgs:
                alpha_remove[i] = self.alpha(g_co,rmv_edge)
                i+=1

            idx_remove   = np.argmin(alpha_remove)        # index for edge with minimum alpha
            min_rmv_edge = rmv_edgs[idx_remove]           # edge with minimum alpha
            g_co.remove_edge(*min_rmv_edge)

            if not(nx.is_connected(g_co)):
                g_co.add_edge(*min_rmv_edge)
                j-=1
            else:
                i = 0
                for add_edge in add_edgs:
                    alpha_add[i] = self.alpha(g_co,add_edge)
                    i+=1

                idx_add    = np.argmax(alpha_add)         # index for edge with maximum alpha
                max_add_edge = add_edgs[idx_add]          # edge with maximum alpha
                g_co.add_edge(*max_add_edge)

                rmv_edgs = rmv_edgs[np.setdiff1d(np.arange(0,rmv_edgs.shape[0]) , idx_remove)]
                add_edgs = add_edgs[np.setdiff1d(np.arange(0,add_edgs.shape[0]) , idx_add   )]    
            j+=1
        return g_co
    
    def add_edge(self):
        g_co = self.g.copy()
        g_co_c = self.g_c.copy()
        add_edgs = np.array(g_co_c.edges() , dtype='int32')
        
        j=0
        while (j < self.m):
            alpha_add= np.zeros(add_edgs.shape[0])   # array for add edge alpha

            i = 0
            for add_edge in add_edgs:
                alpha_add[i] = self.alpha(g_co , add_edge)
                i+=1

            idx_add= np.argmax(alpha_add)    # index for edge with maximum alpha
            
            max_add_edge= add_edgs[idx_add]     # edge with maximum alpha
           
            g_co.add_edge(*max_add_edge)
            add_edgs = add_edgs[np.setdiff1d(np.arange(0,add_edgs.shape[0]) , idx_add)]    
            j+=1
        return g_co
    
    def compute_Algebraic_connectivity(self, A):
        ei  =np.linalg.eigvalsh(A)
        alg =ei[1]
        return alg
    
    def evalute_graph(self,func = 'OptimizeAlgebraicSydneyFirstRmv'):
        g_co = getattr(self,func)()
        L = nx.laplacian_matrix(g_co).todense()
        return self.compute_Algebraic_connectivity(L) , g_co